In [ ]:
!task helm:build CHART=apache-jena

In [ ]:
!task helm:install CHART=apache-jena

In [ ]:
!task helm:delete CHART=apache-jena

In [ ]:
!pip install SPARQLWrapper

In [41]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [42]:
sparql_endpoint = "http://localhost:3030/ds"

sparql = SPARQLWrapper(sparql_endpoint)

In [43]:
def insert_data():
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        INSERT DATA {
            ex:subject1 rdf:type ex:Person ;
                         ex:name "John" .

            ex:subject2 rdf:type ex:Person ;
                         ex:name "Jane" .
        }
    """

    sparql.setQuery(sparql_query)
    sparql.setMethod('POST')

    sparql.query()

    print("Data inserted successfully.")

In [44]:
def search_data(sparql_query=None):
    if sparql_query is None:

        sparql_query = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX ex: <http://example.org/>

            SELECT ?name
            WHERE {
                ?subject rdf:type ex:Person ;
                         ex:name ?name .
            }
        """

    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()

    if 'results' in results:
        bindings = results['results']['bindings']
        if len(bindings) > 0:
            print("Results:")
            for binding in bindings:
                name = binding['name']['value']
                print("Name:", name)
        else:
            print("No results found.")
    else:
        print("No results found.")

In [45]:
def search_data_by_name(name):
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        SELECT ?name
        WHERE {
            ?subject rdf:type ex:Person ;
                     ex:name ?name .
            FILTER (str(?name) = "%s")
        }
    """ % name
    search_data(sparql_query)

In [46]:
def update_data(subject_uri, new_name):
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        DELETE {{
            <{subject_uri}> ex:name ?old_name .
        }}
        INSERT {{
            <{subject_uri}> ex:name "{new_name}" .
        }}
        WHERE {{
            <{subject_uri}> rdf:type ex:Person ;
                            ex:name ?old_name .
        }}
    """.format(subject_uri=subject_uri, new_name=new_name)

    sparql.setQuery(sparql_query)
    sparql.setMethod('POST')

    sparql.query()

    print("Data updated successfully.")

In [47]:
insert_data()

Data inserted successfully.


In [48]:
search_data()

Results:
Name: John
Name: John Doe
Name: Jane


In [49]:
search_data_by_name("John")

Results:
Name: John


In [50]:
update_data("http://example.org/subject1", "John Doe")

Data updated successfully.


In [51]:
search_data()

Results:
Name: John Doe
Name: Jane


In [52]:
search_data_by_name("John Doe")

Results:
Name: John Doe
